# Homework 10
## ECE 204 Data Science & Engineering

In [42]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings("ignore")

# Add other import statements you need here
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

---
**Problem 1.** Read in `australia-rosslyn-bay.csv`.

**What is the mean air temperature for all measurements made between 2 pm and 6 pm (Both inclusive)?**

*Note:* in 24-hour time (which is what this dataset uses, 2pm is hour 14 and 6pm is hour 18).

In [35]:
# given code
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

# your code here
df.head()

new_df = df[(df.index.hour >= 14) & (df.index.hour <= 18)]
new_df['air_temp'].mean()


21.59373263605164

---
**Problem 2.** Read in `australia-rosslyn-bay.csv`. **What is the mean air temperature for all measurements made in October?**

In [37]:
# given code
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

# your code here
new_df = df[(df.index.month == 10)]
new_df['air_temp'].mean()


23.13707997641973

---
**Problem 3.** Read in `australia-rosslyn-bay.csv`. **What month has the lowest average pressure?** Report the answer as a number (e.g., "3" for "March").

In [36]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

# your code here
for i in range(1,11):
    new_df = df[(df.index.month == i)]
    print(new_df['pressure'].mean())




1009.8969990974729
1010.1980465815176
1012.500877770463
1016.0871671671672
1016.4928805710978
1018.7389716100854
1019.1519600307455
1018.5435132482825
1017.57908017908
1014.5137355079582


---
**Problem 4.** Read in australia-rosslyn-bay.csv. **What is the 95th percentile (or quantile) of the air pressure during August?**

*Hint:* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.quantile.html

In [39]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

# your code here
new_df = df[(df.index.month == 8)]
new_df['pressure'].quantile(0.95)


1024.0

---

The following questions are based on a synthetic dataset "share-prices.csv" that capture daily mean share prices ((opening + closing)/ 2) of a company from 1st January, 2000 to 2nd May, 2000. This amount to 123 mean share price observations.

The next two questions will pretend you're a stock market analyst on May 2nd, 2000 and are developing a new method of finding future stock prices. You're using all days up to and including April 19th, 2000 as train data, and trying to predict stock prices on April 20th, 2000 and beyond.

Let's pretend we're developing a new technique to predict future performance. This means that there is a natural division between the train and test sets:

- **Train set:** January 1st, 2000 (2000-01-01) to 19th April, 2000 (2000-04-19), or 110 days.
- **Test set:** April 20th, 2000 (2000-04-20) to the end, 2nd May, 2000 (2000-05-02), or 13 days.

The next two questions use the library `statsmodels`, and only for the "auto-regressive" model or AR model. Auto-regression is linear regression on the last `maxlag` values of a time series (and below the statsmodel package is only used to help with predicting more than one new value).

`statsmodels` differs from Scikit-Learn in a couple ways:

* In statsmodels, the data is passed to during model creation (not during model fitting like Scikit-Learn).
* In statsmodels, the hyper-parameters are passed to during model fitting (not during model creation like Scikit-Learn).

Notably, predicting new values only needs the train dataset: the prediction works recursively starting from the last value in the train dataset.

---
**Problem 5.** Read in `share-prices.csv`. Train an auto-regressive model with the previous 4 weeks of data, or `lags=28`; that is, predict today's stock price given the last 28 days.

Use the historical data (or train set) to fit the model. Predict stock price from April, 20th 2000 to May 2nd, 2000. This will happen recursively using statsmodels – the statsmodel auto-regressive model will generate a prediction for April 20th, then use that value to predict April 21st and so on.

**What is the mean squared error between the predicted and true values from April 20th to May 2nd 2000?**

*Gold star question:* Would you use this model for actually making decisions on stock transactions? It'd be useful to plot and visualize both the predicted and true values for the testing time-period. 

In [40]:
df = pd.read_csv("share_prices.csv", parse_dates=True, index_col=0)
df.index.freq = "D"  # set as "daily" frequency; observations are recorded every day. Removes a warning in statsmodels

# Training and testing set
df_train = df.loc[:'2000-04-19']
df_test = df.loc['2000-04-20':]
df_train.tail()

,price
date,
2000-04-15,57.24
2000-04-16,42.70
2000-04-17,51.28
2000-04-18,58.82
2000-04-19,64.79


In [47]:
# your code here
df = pd.read_csv("share_prices.csv", parse_dates=True, index_col=0)
df.index.freq = "D"  # set as "daily" frequency; observations are recorded every day. Removes a warning in statsmodels

# Training and testing set
df_train = df.loc[:'2000-04-19']
df_test = df.loc['2000-04-20':]


model = AutoReg(df_train, lags=28)
model_fit = model.fit()

predictions = model_fit.predict(start='2000-04-20', end='2000-05-02')

mean_squared_error(df_test, predictions)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


1189.408002180309

---
**Problem 6.** In this question, **use auto-regression model to find the best value of `lags` from 1 to 10 both inclusive**. Here **best** is defined to be **"minimum mean squared error on the validation set"**. The dataset has been split as follows:

- Training set: January 1st, 2000 to April 1st, 2000
- Validation set: from April 2nd, 2000 to May 2nd, 2000.

The validation set has 31 observations, which provides a more fair assessment of different values of `maxlag`; there are at least 20 predictions that are solely created by the model and don't use any "free" historical data.

*Gold star question: Plot the predictions for the best `lags`. Would you use this model to buy stocks?*

In [48]:
df = pd.read_csv("share_prices.csv", parse_dates=True, index_col=0)
df.index.freq = "D"  # set as "daily" frequency; observations are recorded every day. Removes a warning in statsmodels

# Training and validation set
df_train = df.loc[:'2000-04-01']
df_val = df.loc['2000-04-02':]

In [56]:
# your code here

def evaluate_model(maxlag, train, val):
    model = AutoReg(train, lags=maxlag)
    model_fit = model.fit()
    predictions = model_fit.predict(start=val.index[0], end=val.index[-1])
    mse = mean_squared_error(val, predictions)
    return mse

best_mse = float('inf')
best_maxlag = None
for maxlag in range(1, 11):
    mse = evaluate_model(maxlag, df_train, df_val)
    if mse < best_mse:
        best_mse = mse
        best_maxlag = maxlag

best_maxlag


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


6

In [59]:
A = np.array([[17,  6, 20, -2, -9,  3, 18, 14,  2],
       [ 2, 17,  6, 20, -2, -9,  3, 18, 14],
       [14,  2, 17,  6, 20, -2, -9,  3, 18],
       [18, 14,  2, 17,  6, 20, -2, -9,  3],
       [ 3, 18, 14,  2, 17,  6, 20, -2, -9],
       [-9,  3, 18, 14,  2, 17,  6, 20, -2],
       [-2, -9,  3, 18, 14,  2, 17,  6, 20],
       [20, -2, -9,  3, 18, 14,  2, 17,  6],
       [ 6, 20, -2, -9,  3, 18, 14,  2, 17]])


for i in range(1, len(A)):
    if not np.array_equal(A[i], np.roll(A[i-1], 1)):
        print("No")
        break



In [61]:


B = np.array([[17,  2, 14, 18,  3, -9, -2, 20,  6,  5,  2],
              [18,  3, -9, -2, 20,  6,  5,  2, 17,  2, 14],
              [-2, 20,  6,  5,  2, 17,  2, 14, 18,  3, -9],
              [ 5,  2, 17,  2, 14, 18,  3, -9, -2, 20,  6],
              [ 2, 14, 18,  3, -9, -2, 20,  6,  5,  2, 17],
              [ 3, -9, -2, 20,  6,  5,  2, 17,  2, 14, 18],
              [20,  6,  5,  2, 17,  2, 14, 18,  3, -9, -2],
              [ 2, 17,  2, 14, 18,  3, -9, -2, 20,  6,  5],
              [14, 18,  3, -9, -2, 20,  6,  5,  2, 17,  2],
              [-9, -2, 20,  6,  5,  2, 17,  2, 14, 18,  3],
              [ 6,  5,  2, 17,  2, 14, 18,  3, -9, -2, 20]])


for i in range(1, len(B)):
    if not np.array_equal(B[i], np.roll(B[i-1], -3)):
        print("No")
        break



In [64]:
s = 'Feature engineering involves creating new features from existing data to improve model performance.'

total_sum = 0
for char in s:
    total_sum += ord(char)
total_sum

9650

In [70]:
s = 'AUC-ROC curve measures the performance of binary classification models.'
total_sum = 0

for char in s:
    total_sum += ord(char)
result = total_sum % 23

result


17

In [80]:



expressions = [
    "Data analysis is essential for extracting insights from raw data.",
    "Machine learning algorithms play a crucial role in predictive modeling.",
    "Statistical modeling helps us understand relationships between variables.",
    "Predictive analytics enables forecasting future outcomes based on historical data.",
    "Big data technologies handle massive datasets efficiently.",
    "Data visualization tools create informative visual representations of data.",
    "Natural language processing techniques analyze and process human language data.",
    "Feature engineering enhances model performance by creating new features.",
    "Exploratory data analysis identifies patterns and anomalies in data.",
    "Regression analysis is used to model relationships between variables.",
    "Clustering algorithms group similar data points together.",
    "Dimensionality reduction techniques reduce the number of features in data.",
    "Time series analysis focuses on temporal patterns in data.",
    "Cross-validation assesses model performance on unseen data.",
    "Ensemble methods combine multiple models for better predictions.",
    "Hypothesis testing evaluates statistical significance in experiments.",
    "AUC-ROC curve measures the performance of binary classification models.",
    "Precision and recall are important metrics for evaluating model performance.",
    "Overfitting occurs when a model performs well on training data but poorly on test data.",
    "Bias-variance trade-off balances model complexity and generalization.",
    "Feature selection methods choose relevant features for model training."
]

import re


text = ' '.join(expressions)
words = re.findall(r'\b\w{3,}\b', text.lower())
unique_words = sorted(set(words))
word_to_id = {word: idx for idx, word in enumerate(unique_words)}
    



unique_words, word_to_id = find_unique_words(expressions)

text = ' '.join(expressions)

words = re.findall(r'\b\w{3,}\b', text.lower())
nth_word = words[61]

unique_words.index(nth_word)


['algorithms',
 'analysis',
 'analytics',
 'analyze',
 'and',
 'anomalies',
 'are',
 'assesses',
 'auc',
 'balances',
 'based',
 'better',
 'between',
 'bias',
 'big',
 'binary',
 'but',
 'choose',
 'classification',
 'clustering',
 'combine',
 'complexity',
 'create',
 'creating',
 'cross',
 'crucial',
 'curve',
 'data',
 'datasets',
 'dimensionality',
 'efficiently',
 'enables',
 'engineering',
 'enhances',
 'ensemble',
 'essential',
 'evaluates',
 'evaluating',
 'experiments',
 'exploratory',
 'extracting',
 'feature',
 'features',
 'focuses',
 'for',
 'forecasting',
 'from',
 'future',
 'generalization',
 'group',
 'handle',
 'helps',
 'historical',
 'human',
 'hypothesis',
 'identifies',
 'important',
 'informative',
 'insights',
 'language',
 'learning',
 'machine',
 'massive',
 'measures',
 'methods',
 'metrics',
 'model',
 'modeling',
 'models',
 'multiple',
 'natural',
 'new',
 'number',
 'occurs',
 'off',
 'outcomes',
 'overfitting',
 'patterns',
 'performance',
 'performs',


In [98]:
import re
expressions = [
    "Data analysis is essential for extracting insights from raw data.",
    "Machine learning algorithms play a crucial role in predictive modeling.",
    "Statistical modeling helps us understand relationships between variables.",
    "Predictive analytics enables forecasting future outcomes based on historical data.",
    "Big data technologies handle massive datasets efficiently.",
    "Data visualization tools create informative visual representations of data.",
    "Natural language processing techniques analyze and process human language data.",
    "Feature engineering enhances model performance by creating new features.",
    "Exploratory data analysis identifies patterns and anomalies in data.",
    "Regression analysis is used to model relationships between variables.",
    "Clustering algorithms group similar data points together.",
    "Dimensionality reduction techniques reduce the number of features in data.",
    "Time series analysis focuses on temporal patterns in data.",
    "Cross-validation assesses model performance on unseen data.",
    "Ensemble methods combine multiple models for better predictions.",
    "Hypothesis testing evaluates statistical significance in experiments.",
    "AUC-ROC curve measures the performance of binary classification models.",
    "Precision and recall are important metrics for evaluating model performance.",
    "Overfitting occurs when a model performs well on training data but poorly on test data.",
    "Bias-variance trade-off balances model complexity and generalization.",
    "Feature selection methods choose relevant features for model training."
]



text = ' '.join(expressions)
words = re.findall(r'\b\w{3,}\b', text)
unique_words = sorted(set(words))
word_to_id = {word: idx for idx, word in enumerate(unique_words)}
print(unique_words)

nth_word = words[62-1] 
unique_words.index(nth_word)






['AUC', 'Bias', 'Big', 'Clustering', 'Cross', 'Data', 'Dimensionality', 'Ensemble', 'Exploratory', 'Feature', 'Hypothesis', 'Machine', 'Natural', 'Overfitting', 'Precision', 'Predictive', 'ROC', 'Regression', 'Statistical', 'Time', 'algorithms', 'analysis', 'analytics', 'analyze', 'and', 'anomalies', 'are', 'assesses', 'balances', 'based', 'better', 'between', 'binary', 'but', 'choose', 'classification', 'combine', 'complexity', 'create', 'creating', 'crucial', 'curve', 'data', 'datasets', 'efficiently', 'enables', 'engineering', 'enhances', 'essential', 'evaluates', 'evaluating', 'experiments', 'extracting', 'features', 'focuses', 'for', 'forecasting', 'from', 'future', 'generalization', 'group', 'handle', 'helps', 'historical', 'human', 'identifies', 'important', 'informative', 'insights', 'language', 'learning', 'massive', 'measures', 'methods', 'metrics', 'model', 'modeling', 'models', 'multiple', 'new', 'number', 'occurs', 'off', 'outcomes', 'patterns', 'performance', 'performs', 

75

In [106]:
import math

word_frequency = {word: 0 for word in unique_words}
for expression in expressions:
    for word in unique_words:
        if word in expression.lower(): 
            word_frequency[word] += 1
length_expressions = len(expressions)
expression_A = {}
for word, frequency in word_frequency.items():
    A = math.log((length_expressions + 1) / (frequency + 1))
    expression_A[word] = A
expression_B = {word: round(A + 1, 3) for word, A in expression_A.items()}
    
word = unique_words[73]
expression_B[word]

2.992